<a href="https://colab.research.google.com/github/apollonov-evgeniy/test-job/blob/main/test_job.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
import pandas as pd
import os
import shutil
import sqlite3
import re
from datetime import datetime 

In [2]:
conn = sqlite3.connect('tnved.db')
cursor = conn.cursor()

url = "https://www.nalog.gov.ru/rn77/program/5961290/"
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
extract_dir = "./loads"
filename = 'TNVED.zip'
archive_format = 'zip'

test_date  = '2022-09-30'
test_code = '0102299900'

In [3]:
def extract_url():
  #response = requests.get(url, headers=headers)
  #html = response.text

  #soup = BeautifulSoup(html, "html.parser")
  #links = soup.findAll(class_="download")

  #links = soup.findAll("a")
  #for link in links:
  #    print(link.get('href'))
  download_url = 'https://www.nalog.gov.ru/html/sites/www.new.nalog.ru/docs/sprav/tnved/TNVED.ZIP'

  download_response = requests.get(download_url, headers=headers)
  with open('TNVED.zip', 'wb') as f: 
      f.write(download_response.content)
  shutil.unpack_archive(filename, extract_dir, archive_format)    
  os.remove(filename) 
  files_received = os.listdir(extract_dir)
  for i in files_received:
    os.rename(os.path.join(extract_dir, i), os.path.join(extract_dir, i.lower()) ) 
  files_received = os.listdir(extract_dir) 
  return  files_received  

In [4]:
def select(sql):
  return pd.read_sql(sql,conn) 

def drop_all_table_sqllite(): # удаление таблиц из базы sqllite, если уже была заполнена
  sql = '''SELECT name FROM sqlite_master WHERE type= "table";'''
  fg = select(sql)

  for i in fg['name']:
    if i != 'sqlite_sequence':
      sql = f'''DROP TABLE IF EXISTS "{i}"  
      '''
      conn.execute(sql)
  conn.commit()

In [5]:
def init_tables_dwh(): # cоздание схемы DWH sqllite
	sql_great_db ='''
	CREATE TABLE if not exists dwh_dim_sections_hist (
    id             INTEGER     PRIMARY KEY AUTOINCREMENT,
    section_nbr    TEXT (2)    NOT NULL,
    section_name   TEXT (4000),
    section_note   TEXT (4000),
    effective_from DATE,
    effective_to   DATE        DEFAULT '2199-12-31',
    -- deleted_flg    INTEGER     DEFAULT (0),
    UNIQUE (
        section_nbr,
        effective_from
    )
	);
	CREATE TABLE if not exists dwh_dim_groups_hist (
    id             INTEGER     PRIMARY KEY AUTOINCREMENT,
    section_nbr    TEXT (2)    NOT NULL,
    group_nbr      TEXT (2)    NOT NULL,
    group_name     TEXT (4000),
    group_note     TEXT (4000),
    effective_from DATE,
    effective_to   DATE        DEFAULT '2199-12-31',
    -- deleted_flg    INTEGER     DEFAULT (0),
    UNIQUE (
        group_nbr,
        effective_from
    ),
    FOREIGN KEY (
        section_nbr
    )
    REFERENCES dwh_dim_sections_hist (section_nbr) 
	);
	CREATE TABLE if not exists dwh_dim_items_hist (
    id             INTEGER     PRIMARY KEY AUTOINCREMENT,
    group_nbr      TEXT (2)    NOT NULL,
    item_nbr       TEXT (2)    NOT NULL,
    item_name      TEXT (4000),
    effective_from DATE,
    effective_to   DATE        DEFAULT '2199-12-31',
    -- deleted_flg    INTEGER     DEFAULT (0),
    UNIQUE (
        group_nbr,
				item_nbr,
        effective_from
    ),
    FOREIGN KEY (
        group_nbr
    )
    REFERENCES dwh_dim_groups_hist (group_nbr) 
	);
	CREATE TABLE if not exists dwh_dim_subitems_hist (
			id             INTEGER     PRIMARY KEY AUTOINCREMENT,
			group_nbr      TEXT (2)    NOT NULL,
			item_nbr       TEXT (2)    NOT NULL,
			subitem_nbr    TEXT (6)    NOT NULL,
			subitem_name   TEXT (4000),
			effective_from DATE,
			effective_to   DATE        DEFAULT ('2199-12-31'),
			UNIQUE (
					group_nbr,
					item_nbr,
					subitem_nbr,
					effective_from
			),
			FOREIGN KEY (
					group_nbr,
					item_nbr
			)
			REFERENCES dwh_dim_items_hist (group_nbr,
			item_nbr) 
	);
	'''
	cursor.executescript(sql_great_db)
	conn.commit()

In [6]:
def tnved1_transfer():
  df_sections = pd.read_csv('./loads/tnved1.txt', 
                          sep ='|', 
                          header = None,
                          skiprows=[0],
                          encoding = 'cp866',
                          index_col = False,
                          usecols=[0, 1, 2 , 3, 4],
                          dtype = {0: str},
                          )
  df_sections[3] = pd.to_datetime(df_sections[3], format='%d.%m.%Y')
  df_sections[4] = df_sections[4].fillna('31.12.2199')
  df_sections[4] = pd.to_datetime(df_sections[4], format='%d.%m.%Y')
  column_names = ['section_nbr','section_name','section_note','effective_from','effective_to']
  df_sections.columns = column_names 
  df_sections.to_sql('stg_sections', conn, if_exists='replace', index = False)

def tnved1_great_dwh(): # STG --> HIST    
    sql_great_section = f''' 
INSERT INTO dwh_dim_sections_hist (section_nbr, section_name, section_note, effective_from, effective_to)
SELECT t1.section_nbr,
       t1.section_name,
       t1.section_note,
       date(t1.effective_from),
       date(t1.effective_to)
FROM stg_sections t1
EXCEPT
SELECT t2.section_nbr,
       t2.section_name,
       t2.section_note,
       date(t2.effective_from),
       date(t2.effective_to)
FROM dwh_dim_sections_hist t2;

DROP TABLE IF EXISTS stg_sections;'''
    cursor.executescript(sql_great_section)
    conn.commit()    

    print('tnved1.txt загружено и обработано')  


In [7]:
def tnved2_transfer():
  df_groups = pd.read_csv('./loads/tnved2.txt', 
                          sep ='|', 
                          header = None,
                          skiprows=[0],
                          encoding = 'cp866',
                          index_col = False,
                          usecols=[0, 1, 2 , 3, 4, 5],
                          dtype = {0: str, 1:str},
                          )
  df_groups[4] = pd.to_datetime(df_groups[4], format='%d.%m.%Y')
  df_groups[5] = df_groups[5].fillna('31.12.2199')  
  df_groups[5] = pd.to_datetime(df_groups[5], format='%d.%m.%Y')
  column_names = ['section_nbr', 'group_nbr', 'group_name','group_note','effective_from','effective_to']
  df_groups.columns = column_names 
  df_groups.to_sql('stg_groups', conn, if_exists='replace', index = False)

def tnved2_great_dwh(): # STG --> HIST    
    sql_great_groups = f''' 
    INSERT INTO dwh_dim_groups_hist (section_nbr, group_nbr, group_name, group_note, effective_from, effective_to)
    SELECT t1.section_nbr,
          t1.group_nbr,
          t1.group_name,
          t1.group_note,
          date(t1.effective_from),
          date(t1.effective_to)
    FROM stg_groups t1
    EXCEPT
    SELECT t2.section_nbr,
          t2.group_nbr,
          t2.group_name,
          t2.group_note,
          date(t2.effective_from),
          date(t2.effective_to)
    FROM dwh_dim_groups_hist t2;

    DROP TABLE IF EXISTS stg_groups;'''
    cursor.executescript(sql_great_groups)
    conn.commit()    

    print('tnved2.txt загружено и обработано') 

In [8]:
def tnved3_transfer():
  df_items = pd.read_csv('./loads/tnved3.txt', 
                          sep ='|', 
                          header = None,
                          skiprows=[0],
                          encoding = 'cp866',
                          index_col = False,
                          usecols=[0, 1, 2 , 3, 4],
                          dtype = {0: str, 1:str},
                          )
  df_items[3] = pd.to_datetime(df_items[3], format='%d.%m.%Y')
  df_items[4] = df_items[4].fillna('31.12.2199')  
  df_items[4] = pd.to_datetime(df_items[4], format='%d.%m.%Y')
  column_names = ['group_nbr', 'item_nbr', 'item_name','effective_from','effective_to']
  df_items.columns = column_names
  df_items.to_sql('stg_items', conn, if_exists='replace', index = False) 

def tnved3_great_dwh(): # STG --> HIST    
    sql_great_items = f''' 
    INSERT INTO dwh_dim_items_hist (group_nbr, item_nbr, item_name, effective_from, effective_to)
    SELECT t1.group_nbr,
          t1.item_nbr,
          t1.item_name,
          date(t1.effective_from),
          date(t1.effective_to)
    FROM stg_items t1
    EXCEPT
    SELECT t2.group_nbr,
          t2.item_nbr,
          t2.item_name,
          date(t2.effective_from),
          date(t2.effective_to)
    FROM dwh_dim_items_hist t2;

    DROP TABLE IF EXISTS stg_items;'''
    cursor.executescript(sql_great_items)
    conn.commit()    

    print('tnved3.txt загружено и обработано')

In [9]:
def tnved4_transfer():
  df_subitems = pd.read_csv('./loads/tnved4.txt', 
                        sep ='|', 
                        header = None,
                        skiprows=[0],
                        encoding = 'cp866',
                        index_col = False,
                        usecols=[0, 1, 2 , 3, 4, 5],
                        dtype = {0: str, 1:str, 3: str},
                        )
  df_subitems[4] = pd.to_datetime(df_subitems[4], format='%d.%m.%Y')
  df_subitems[5] = df_subitems[5].fillna('31.12.2199')
  df_subitems[5] = pd.to_datetime(df_subitems[5], format='%d.%m.%Y')
  column_names = ['group_nbr', 'item_nbr', 'subitem_nbr', 'subitem_name','effective_from','effective_to']
  df_subitems.columns = column_names 
  df_subitems.to_sql('stg_subitems', conn, if_exists='replace', index = False)

def tnved4_great_dwh(): # STG --> HIST    
    sql_great_subitems = f''' 
    INSERT INTO dwh_dim_subitems_hist (group_nbr, item_nbr, subitem_nbr, subitem_name, effective_from, effective_to)
    SELECT t1.group_nbr,
          t1.item_nbr,
          t1.subitem_nbr,
          t1.subitem_name,
          date(t1.effective_from),
          date(t1.effective_to)
    FROM stg_subitems t1
    EXCEPT
    SELECT t2.group_nbr,
          t2.item_nbr,
          t2.subitem_nbr,
          t2.subitem_name,
          date(t2.effective_from),
          date(t2.effective_to)
    FROM dwh_dim_subitems_hist t2;

    DROP TABLE IF EXISTS stg_subitems;'''
    cursor.executescript(sql_great_subitems)
    conn.commit()    

    print('tnved4.txt загружено и обработано')  

In [10]:
# main

# подготовка DWH
if input('создать DWH в первый раз y/n  -  ') == 'y':
  drop_all_table_sqllite() 
  init_tables_dwh()
# парсинг сайта, скачивание и распаковка архива
print('распакованы файлы', extract_url()) 

# обработка TXT --> STG --> DWH 
tnved1_transfer()
tnved1_great_dwh()

tnved2_transfer()
tnved2_great_dwh()

tnved3_transfer()
tnved3_great_dwh()

tnved4_transfer()
tnved4_great_dwh()

shutil.rmtree(extract_dir)


создать DWH в первый раз y/n  -  y
распакованы файлы ['tnved2.txt', 'tnved1.txt', 'tnved4.txt', 'tnved3.txt']
tnved1.txt загружено и обработано
tnved2.txt загружено и обработано
tnved3.txt загружено и обработано
tnved4.txt загружено и обработано


### Проверка 

In [11]:
sql ='SELECT name from sqlite_master where type= "table"'
select(sql)

,name
0,sqlite_sequence
1,dwh_dim_sections_hist
2,dwh_dim_groups_hist
3,dwh_dim_items_hist
4,dwh_dim_subitems_hist


In [12]:
sql_test = f'''
select t1.section_nbr as Раздел, 
  t1.section_name Наименование, 
  t2.group_nbr as Группа, 
  t2.group_name Наименование, 
  t3.item_nbr Позиция, 
  t3.item_name Наименование, 
  t4.subitem_nbr Подпозиция, 
  t4.subitem_name Наименование
from dwh_dim_sections_hist t1
    inner join dwh_dim_groups_hist t2
    on t1.section_nbr = t2.section_nbr 
    inner join dwh_dim_items_hist t3 
    on t2.group_nbr = t3.group_nbr 
    inner join dwh_dim_subitems_hist t4
    on t3.item_nbr = t4.item_nbr and t3.group_nbr = t4.group_nbr 
where '{test_date}' between t1.effective_from and t1.effective_to 
and  '{test_date}' between t2.effective_from and t2.effective_to
and  '{test_date}' between t3.effective_from and t3.effective_to
and  '{test_date}' between t4.effective_from and t4.effective_to
and t2.group_nbr = SUBSTR('{test_code}', 1, 2) 
and   t3.item_nbr = SUBSTR('{test_code}', 3, 2) 
and t4.subitem_nbr = SUBSTR('{test_code}', 5, 6);
'''
select(sql_test)


,Раздел,Наименование,Группа,Наименование,Позиция,Наименование,Подпозиция,Наименование
0,01,ЖИВЫЕ ЖИВОТНЫЕ; ПРОДУКТЫ ЖИВОТНОГО ПРОИСХОЖДЕНИЯ,01,ЖИВЫЕ ЖИВОТНЫЕ,02,КРУПНЫЙ РОГАТЫЙ СКОТ ЖИВОЙ,299900,- - - - - - прочие
